In [1]:
# Celda 1: Carga de Librerías y Datos
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split


In [2]:
import numpy as np
import random
import os

def set_seed(seed=1):
    np.random.seed(seed)          # NumPy
    random.seed(seed)             # Python Random
    os.environ['PYTHONHASHSEED'] = str(seed)  # Python hash seed (consistencia)

    try:
        from surprise import SVD
        print("Estableciendo semilla en Surprise...")
        np.random.seed(seed)  # En Surprise usa np.random
    except ImportError:
        pass

set_seed(1)


Estableciendo semilla en Surprise...


In [3]:
train_data = pd.read_csv("data/train.csv", sep=",")
test_data = pd.read_csv("data/test.csv", sep=",")

global_mean = train_data['rating'].mean()
print(f"Media global de ratings: {global_mean:.4f}")

Media global de ratings: 7.6047


In [4]:
train_data.head(5)

,user,item,rating
0,1,25715,7.0
1,1,25716,10.0
2,5,25851,9.0
3,6,25923,5.0
4,7,25924,6.0


In [5]:
test_data.head(5)

,ID,user,item
0,0,8117,268
1,1,10512,24393
2,2,534,1334
3,3,10984,6550
4,4,9093,22128


In [6]:
#train_data, val_split = train_test_split(train_data, test_size=0.2, random_state=42)

In [7]:
# Obtener listas únicas de usuarios e ítems del conjunto de entrenamiento
unique_users = train_data['user'].unique()
unique_items = train_data['item'].unique()

# Crear índices para usuarios e ítems
user_to_index = {user: idx for idx, user in enumerate(unique_users)}
item_to_index = {item: idx for idx, item in enumerate(unique_items)}

# Dimensiones de la matriz de ratings
num_users = len(unique_users)
num_items = len(unique_items)

print(f"Número de usuarios: {num_users}")
print(f"Número de ítems: {num_items}")

Número de usuarios: 73456
Número de ítems: 171171


In [8]:
from scipy.sparse import csr_matrix

# Crear la matriz dispersa solo con el conjunto de entrenamiento
rows = train_data['user'].map(user_to_index).values
cols = train_data['item'].map(item_to_index).values
ratings = train_data['rating'].values

R_sparse = csr_matrix((ratings, (rows, cols)), shape=(num_users, num_items))

print(f"Matriz dispersa creada con dimensiones: {R_sparse.shape}")
print("Número de ratings no nulos en la matriz:", R_sparse.nnz)

Matriz dispersa creada con dimensiones: (73456, 171171)
Número de ratings no nulos en la matriz: 390351


In [9]:
set_seed(1)

# Celda 3: Inicialización de Parámetros y Factores Latentes con Bias
# Hiperparámetros
k = 20  # Número de factores latentes
lambda_ = 0.01  # Regularización
learning_rate = 0.0025  # Tasa de aprendizaje
num_epochs = 50  # Número de iteraciones

# Media global de ratings
mu = global_mean

# Inicialización aleatoria de las matrices latentes U y V
U = np.random.normal(scale=1./k, size=(num_users, k))
V = np.random.normal(scale=1./k, size=(num_items, k))

# Inicialización de sesgos
bu = np.zeros(num_users)
bi = np.zeros(num_items)

print(f"Factores latentes inicializados: U ({U.shape}), V ({V.shape})")
print(f"Sesgos inicializados: bu ({bu.shape}), bi ({bi.shape})")


Estableciendo semilla en Surprise...
Factores latentes inicializados: U ((73456, 20)), V ((171171, 20))
Sesgos inicializados: bu ((73456,)), bi ((171171,))


In [10]:
# Celda 4: Función de Entrenamiento PMF con Bias
def train_pmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, lr, num_epochs):
    rows, cols = R_sparse.nonzero()
    num_ratings = len(rows)

    for epoch in range(num_epochs):
        total_cost = 0
        for idx in range(num_ratings):
            i = rows[idx]  # Usuario
            j = cols[idx]  # Ítem
            r_ij = R_sparse[i, j]

            # Predicción incluyendo el sesgo
            pred_ij = mu + bu[i] + bi[j] + np.dot(U[i, :], V[j, :])
            error = r_ij - pred_ij

            # Actualización de los factores latentes y sesgos
            bu[i] += lr * (error - lambda_ * bu[i])
            bi[j] += lr * (error - lambda_ * bi[j])
            U[i, :] += lr * (error * V[j, :] - lambda_ * U[i, :])
            V[j, :] += lr * (error * U[i, :] - lambda_ * V[j, :])

            # Costo regularizado
            total_cost += error**2 + (lambda_ / 2) * (np.linalg.norm(U[i, :])**2 + np.linalg.norm(V[j, :])**2 + bu[i]**2 + bi[j]**2)

        print(f"Época {epoch + 1}/{num_epochs} - Costo total: {total_cost:.4f}")

    return U, V, bu, bi

In [11]:
# Celda 5: Entrenamiento del Modelo PMF
print("Iniciando entrenamiento PMF...")
U, V, bu, bi = train_pmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, learning_rate, num_epochs)
print("Entrenamiento completado.")


Iniciando entrenamiento PMF...
Época 1/50 - Costo total: 1252112.2893
Época 2/50 - Costo total: 1179491.7264
Época 3/50 - Costo total: 1139154.8887
Época 4/50 - Costo total: 1109724.2523
Época 5/50 - Costo total: 1086067.9612
Época 6/50 - Costo total: 1066034.1274
Época 7/50 - Costo total: 1048487.5476
Época 8/50 - Costo total: 1032742.5712
Época 9/50 - Costo total: 1018338.3200
Época 10/50 - Costo total: 1004929.4995
Época 11/50 - Costo total: 992224.7303
Época 12/50 - Costo total: 979950.2713
Época 13/50 - Costo total: 967839.5536
Época 14/50 - Costo total: 955666.7858
Época 15/50 - Costo total: 943344.7203
Época 16/50 - Costo total: 931048.7257
Época 17/50 - Costo total: 919208.7601
Época 18/50 - Costo total: 908241.4476
Época 19/50 - Costo total: 898248.6917
Época 20/50 - Costo total: 889032.9117
Época 21/50 - Costo total: 880324.4149
Época 22/50 - Costo total: 871919.6835
Época 23/50 - Costo total: 863690.8177
Época 24/50 - Costo total: 855557.5122
Época 25/50 - Costo total: 84746

In [12]:
# # Celda 6: Evaluación del Modelo (MAE)
# def calculate_mae(U, V, data):
#     total_error = 0
#     count = 0
#     for _, row in data.iterrows():
#         user_id = row['user']
#         item_id = row['item']
#         rating = row['rating']
#         if user_id in user_to_index and item_id in item_to_index:
#             pred = np.dot(U[user_to_index[user_id], :], V[item_to_index[item_id], :])
#             error = abs(rating - pred)
#             total_error += error
#             count += 1
#     mae = total_error / count if count > 0 else float('inf')
#     return mae

# # Calcular MAE en el conjunto de validación
# mae = calculate_mae(U, V, val_split)
# print(f"MAE en el conjunto de validación: {mae:.4f}")

In [13]:
from surprise import SVD, Dataset, Reader

# Fijar el seed para el modelo SVD
set_seed(1)

# Crear el modelo SVD
svd = SVD(n_factors=20, n_epochs=50, biased=True, lr_all=0.002, reg_all=0.02, random_state=42)

# Cargar el conjunto de entrenamiento para Surprise
reader = Reader(rating_scale=(1, 10))
trainset = Dataset.load_from_df(train_data[['user', 'item', 'rating']], reader).build_full_trainset()

# Entrenamiento del modelo SVD
print("Entrenando el modelo SVD...")
svd.fit(trainset)
print("Modelo SVD entrenado correctamente.")

Estableciendo semilla en Surprise...
Entrenando el modelo SVD...
Modelo SVD entrenado correctamente.


In [ ]:
# Celda 9: Predicción combinada ponderada con PMF (con bias) y SVD
def predict_rating(uid, iid, U, V, bu, bi, mu):
    try:
        # Predicción usando SVD
        pred_svd = svd.predict(uid, iid).est
    except:
        pred_svd = None

    # Predicción usando PMF con bias
    if uid in user_to_index and iid in item_to_index:
        user_idx = user_to_index[uid]
        item_idx = item_to_index[iid]
        pred_pmf = mu + bu[user_idx] + bi[item_idx] + np.dot(U[user_idx, :], V[item_idx, :])
    else:
        pred_pmf = None

    # Ponderación de las predicciones (70% SVD y 30% PMF)
    if pred_svd is not None and pred_pmf is not None:
        pred = (0.4 * pred_svd) + (0.6 * pred_pmf)
    elif pred_svd is not None:
        pred = pred_svd
    elif pred_pmf is not None:
        pred = pred_pmf
    else:
        pred = mu  # Si ambos fallan, usar la media global

    # Clamping entre 1 y 10 con formato decimal ".0"
    return f"{round(max(1, min(10, pred)))}.0"

In [19]:
# Celda 10: Generación del Archivo de Predicciones
print("Generando predicciones finales...")
predictions = []

for _, row in test_data.iterrows():
    uid, iid, row_id = row['user'], row['item'], row['ID']
    pred_rating = predict_rating(uid, iid, U, V, bu, bi, mu)
    predictions.append((row_id, pred_rating))

# Crear el DataFrame con las predicciones
predictions_df = pd.DataFrame(predictions, columns=["ID", "rating"])

# Guardar las predicciones en formato CSV
output_filename = "predictions_pmf_svd_2.csv"
predictions_df.to_csv(output_filename, index=False)
print(f"Archivo '{output_filename}' generado correctamente.")

Generando predicciones finales...
Archivo 'predictions_pmf_svd_2.csv' generado correctamente.
